# Residual network

This notebook trains the ResNet-20 based on


After training, model is serialized and uploaded to W&B project.

In [11]:
import wandb
import tensorflow as tf
import numpy as np
import pathlib
import shutil

def load_data(run : wandb.sdk.wandb_run.Run) -> pathlib.Path:
    """
    Unpacks data from an artifact into a folder and returns the path to the folder.
    """

    artifact_name = f"letters_splits"
    artifact = run.use_artifact(f"master-thesis/{artifact_name}:latest")
    artifact_dir = pathlib.Path(f"./artifacts/{artifact.name.replace(':', '-')}").resolve()
    if not artifact_dir.exists():
        artifact_dir = artifact.download()
        artifact_dir = pathlib.Path(artifact_dir).resolve()
        for split_file in artifact_dir.iterdir():
            if split_file.name.endswith(".tar.gz"):
                split = split_file.name.replace(".tar.gz", "")
                shutil.unpack_archive(split_file, artifact_dir / split, format="gztar")
                
    return [ artifact_dir / split for split in ["train", "test", "val"]]

def get_number_of_classes(ds: tf.data.Dataset) -> int:
    """
    Returns the number of classes in a dataset.
    """
    return len(ds.class_names)

def create_tf_dataset(split_path: pathlib.Path, batch_size: int = 32):
    """
    Creates a tf dataset from path containing a folder for each class.
    """
    ds = tf.keras.utils.image_dataset_from_directory(
        split_path, 
        image_size=(32,32), 
        batch_size=batch_size,
        color_mode='grayscale',
    )
    return ds

def preprocess_dataset(ds: tf.data.Dataset, cache: bool = True) -> tf.data.Dataset :
    ds = ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y)) # normalize
    if cache:
        ds = ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

def calculate_model_size_on_disk(path: str) -> int:
    return pathlib.Path(path).stat().st_size    

def calculate_model_num_parameters(model: tf.keras.Model) -> int:
    return model.count_params()

def calculate_model_flops(model: tf.keras.Model) -> str:
    pass

In [15]:
sum([9, 12, 18, 9]) + 2

48

In [12]:
defaults = dict(
    batch_size=32*4,
    epochs=100,    
    optimizer="sgd",
    learning_rate=0.01,
    momentum=0.9,
)

RESNET_DEPTHS = [3, 4, 6, 3]
MODEL_NAME = f"ResNet-{sum(RESNET_DEPTHS) + 2}"
run = wandb.init(project="master-thesis", job_type="training", name=MODEL_NAME, config=defaults,)
split_paths = load_data(run=run)

# hyperparameters

opt_name = wandb.config.optimizer
lr = wandb.config.learning_rate
momentum = wandb.config.momentum
bs = wandb.config.batch_size
epochs = wandb.config.epochs

In [13]:
ds_train, ds_test, ds_val = [
    create_tf_dataset(split_path, batch_size=bs) for split_path in split_paths
    ]

num_classes = len(ds_train.class_names)

print(f"There are {num_classes} classes")
print(f"Training set has {len(ds_train)} batches")
print(f"Test set has {len(ds_test)} batches")
print(f"Validation set has {len(ds_val)} batches")

Found 446491 files belonging to 89 classes.
Found 55773 files belonging to 89 classes.
Found 55773 files belonging to 89 classes.
There are 89 classes
Training set has 3489 batches
Test set has 436 batches
Validation set has 436 batches


In [ ]:
ds_train = preprocess_dataset(ds_train)
ds_val = preprocess_dataset(ds_val)
ds_test = preprocess_dataset(ds_test, cache=False)

In [ ]:
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super(ResidualBlock, self).__init__(**kwargs)
        self.activation = tf.keras.activations.get(activation)
        self.main_layers = [
            tf.keras.layers.Conv2D(filters, strides=strides, kernel_size=3,  padding="same", use_bias=False),
            tf.keras.layers.BatchNormalization(),
            self.activation,
            tf.keras.layers.Conv2D(filters, strides=1, kernel_size=3, padding="same", use_bias=False),
            tf.keras.layers.BatchNormalization()
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                tf.keras.layers.Conv2D(filters, strides=strides, kernel_size=1, padding="same", use_bias=False),
                tf.keras.layers.BatchNormalization()
            ]

    def call(self, inputs):
        x = inputs
        for layer in self.main_layers:
            x = layer(x)
        skip_x = inputs
        for layer in self.skip_layers:
            skip_x = layer(skip_x)
        return self.activation(x + skip_x)

class ResNet(tf.keras.Model):
    def __init__(self, block_design = [3,4,6,3], input_shape=[32, 32, 1], num_classes=88, **kwargs):
        super(ResNet, self).__init__(**kwargs)
        self.num_classes = num_classes
        self.classifier = tf.keras.layers.Dense(num_classes, activation="softmax")
        self.input_conv = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, input_shape=input_shape, padding="same"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Activation("relu"),
            tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same"),
        ])
        self.resnet_blocks = tf.keras.models.Sequential()
        prev_filters = 64
        for filters in np.repeat(np.array([64, 128, 256, 512]), block_design):
            strides = 1 if filters == prev_filters else 2
            self.resnet_blocks.add(ResidualBlock(filters, strides=strides))
            prev_filters = filters
        self.avg_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.flatten = tf.keras.layers.Flatten()
    
    def call(self, inputs):
        x = self.input_conv(inputs)
        x = self.resnet_blocks(x)
        x = self.avg_pool(x)
        x = self.flatten(x)
        return self.classifier(x)

def get_resnet_model(input_shape, block_design, num_classes) -> tf.keras.Sequential:
    model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, input_shape=input_shape, padding="same"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Activation("relu"),
            tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same"),
        ])
    resnet_blocks = tf.keras.models.Sequential()
    for filters in np.repeat(np.array([64, 128, 256, 512]), block_design):
            strides = 1 if filters == prev_filters else 2
            resnet_blocks.add(ResidualBlock(filters, strides=strides))
            prev_filters = filters
    model.add(resnet_blocks)
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(num_classes, activation="softmax"))
    return model

In [ ]:
model = get_resnet_model(input_shape=[32, 32, 1], block_design=RESNET_DEPTHS, num_classes=num_classes)
opt = tf.keras.optimizers.get({
    'class_name': wandb.config.optimizer,
    'config': {
        'learning_rate': lr,
        'momentum': momentum
    }
})

model.compile(
    optimizer=opt,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

wandb_callback = wandb.keras.WandbCallback(
    save_model=False,
    compute_flops=True,
)

# save the best model
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=f"./artifacts/{MODEL_NAME}.h5",
    save_weights_only=False,
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
)

history = model.fit(
    ds_train,
    epochs=epochs,
    validation_data=ds_val,
    callbacks=[wandb_callback, checkpoint_callback],
)

In [ ]:
# plot history
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(15, 10))
epochs = range(1, len(history.history["loss"]) + 1)
ax.plot(epochs, history.history["accuracy"], label="accuracy")
ax.plot(epochs, history.history["val_accuracy"], label="val_accuracy")
ax.set_xlabel("Epoch")
ax.set_ylabel("Accuracy")
ax.legend(loc="lower right")

plt.show()

In [ ]:
# calculate model size on disk, flops and number of parameters

flops = wandb.run.summary["GFLOPs"]
disk_size = calculate_model_size_on_disk(f"./artifacts/{MODEL_NAME}.h5")
num_parameters = calculate_model_num_parameters(model)

# evaluate model on ds_test and log to wandb
test_loss, test_acc = model.evaluate(ds_test)

wandb.log({
    "test loss": test_loss, 
    "test accuracy": test_acc, 
    "number of parameters": num_parameters,
    "disk size": disk_size, 
    "model flops": flops
    })


diacritics = {
    62: "ą",
    63: "ć",
    64: "ę",
    65: "ł",
    66: "ń",
    67: "ó",
    68: "ś",
    69: "ź",
    70: "ż",
    71: "Ą",
    72: "Ć",
    73: "Ę",
    74: "Ł",
    75: "Ń",
    76: "Ó",
    77: "Ś",
    78: "Ź",
    79: "Ż"
}

# log test accuracy on these classes separately to wandb

diacritics_acc = {}
for diacritic_label in diacritics.keys():
    ds_test_diacritic = ds_test.filter(lambda x, y: y == diacritic_label)
    test_loss, test_acc = model.evaluate(ds_test_diacritic)
    diacritics_acc[diacritic_label] = {
        "loss": test_loss,
        "accuracy": test_acc,
        "label": diacritics[diacritic_label],
    }

wandb.log(diacritics_acc)

In [ ]:
# save artifact to wandb
artifact = wandb.Artifact(
    name=MODEL_NAME,
    type="model"
)

# save best model to artifact
artifact.add_file(f"./artifacts/{MODEL_NAME}.h5")
run.log_artifact(artifact)
run.finish()